In [ ]:
import librosa
import numpy
import skimage.io
import os

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def spectrogram_image(y, sr, out, hop_length, n_mels):
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,
                                            n_fft=hop_length*2, hop_length=hop_length)
    mels = numpy.log(mels + 1e-9) # add small number to avoid log(0)

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(numpy.uint8)
    img = numpy.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy

    # save as PNG
    skimage.io.imsave(out, img)

if __name__ == '__main__':
    # settings
    hop_length = 4096 # number of samples per time-step in spectrogram
    n_mels = 105 # number of bins in spectrogram. Height of image
    time_steps = 599 # number of time-steps. Width of image
    
    # extract a fixed length window
    start_sample = 0 # starting at beginning
    length_samples = time_steps*hop_length
    
    
    # load audio. Using example from librosa
    path = "C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/datasets/our_wavs"
    
    os.chdir(path)
    audio_files = os.listdir()
    
    counter = 0 # creating some numbers for new names
    iterate = [-4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7] # helpful array to perform pitch shift augmentation
    
    flat_notes = ['A', 'Bb', 'B', 'C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab']
    sharp_notes = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']
    
    for file in audio_files: # iterate over the wav files 
        
        name, ext = os.path.splitext(file)
        y, sr = librosa.load(file, sr=44100)

        for i in iterate:
            new_name = name
            counter += 1 
            new_name =  str(counter) + new_name 
            
            shifted_y = librosa.effects.pitch_shift(y, sr, n_steps = i)
            
            out = 'C:/outs/our_spects/{0}.png'.format(new_name)
            window = shifted_y[start_sample:start_sample+length_samples]
            # convert to PNG
            spectrogram_image(window, sr=sr, out=out, hop_length=hop_length, n_mels=n_mels)
            
            ann_file = open("C:/outs/24Annotations.txt", "r")
            new_ann = open("C:/outs/shifted24Annotations.txt", "a") # creating new annotation file for shifted spectrograms and the initials 
            
            while True:
                line = ann_file.readline()
                
                if not line:
                    break;
                
                f_id, key, chord = line.split(' ', 2)
                
                f_id = f_id.strip()
                chord = chord.strip()                
                    
                if (f_id == name): # found the file in annotation file 
                    if key in flat_notes:
                        new_ann.write(new_name)
                        new_ann.write(" ")
                        
                        indx = flat_notes.index(key) # index of key in flat notes 
                        
                        if indx+i >= 12: # to avoid segmentation fault 
                            new_key = flat_notes[indx+i-12]
                        else:
                            new_key = flat_notes[indx+i]
                        
                        new_ann.write(new_key)
                        new_ann.write(" ")
                        new_ann.write(chord)
                        new_ann.write("\n")
                        
                    else: # then sharp notes 
                        new_ann.write(new_name)
                        new_ann.write(" ")
                        
                        indx = sharp_notes.index(key) # index of key in sharp notes 
                        if indx+i >= 12: # to avoid segmentation fault 
                            new_key = sharp_notes[indx+i-12]
                        else:
                            new_key = sharp_notes[indx+i]
                        
                        new_ann.write(new_key)
                        new_ann.write(" ")
                        new_ann.write(chord)
                        new_ann.write("\n")
                        
                        
    ann_file.close()
    new_ann.close()
    
    print('wrote file', out)

In [ ]:
# 1st test for pitch_shift algorithm

flat_notes = ['A', 'Bb', 'B', 'C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab']
    
key = 'Bb'
i = -3
#flat_notes[5:].index(key)

if key in flat_notes:
    t = flat_notes.index(key)
    if t+i >= 12:
        a = flat_notes[t+i-12]
    else:
        a = flat_notes[t+i]
    print(a)

In [ ]:
#2nd test for pitch_shift algorithm

counter = 9998 #creating some numbers for new names**
iterate = [-4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7]
new_name = 'kati'

for i in iterate:
    name = new_name
    counter += 1 
    name =  str(counter) + name
    print(name)

In [ ]:
print(X)

In [ ]:
new_ann.close()
ann_file.close()
